In [46]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, Subset, random_split
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torchvision import models
import matplotlib.pyplot as plt
from PIL import Image
import os
import torch.nn.init as init
from sklearn.model_selection import KFold

import warnings
warnings.filterwarnings('ignore')

### Prepare Dataset

In [48]:
class CTScanDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        
        for label, category in enumerate(["Normal", "Covid"]):
            category_path = os.path.join(root_dir, category)
            for img_name in os.listdir(category_path):
                self.image_paths.append(os.path.join(category_path, img_name))
                self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('RGB')
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = CTScanDataset(root_dir='/Users/rohanojha/Documents/01_Sem_1_DS 5220 Code/SML_Project/Code_Test/Train/', transform=transform)
test_dataset   = CTScanDataset(root_dir='/Users/rohanojha/Documents/01_Sem_1_DS 5220 Code/SML_Project/Code_Test/Val/',   transform=transform)

test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

### Model 2 - 4 Layer CNN

In [50]:
''' Independent Channel Processing '''

'''
Changed the number of neurons in the hidden layers
        1. HL1 : 512
        2. HL2 : 256
Max Pool Kernel Size Changed to : 3x3
Using HE Initialization for weights and biases:
        1. Prevents gradient decay in ReLU
        2. Works better for deeper networks(normally greater >3 layers)
        3. Works for ReLU, Leaky ReLU

Xavier Initialization for weights and biases:
        1. Balanced gradients
        2. Works well for shallow networks(normally greater <=3 layers)
        3. Works for Sigmoid, Tanh
'''

class CustomCNN(nn.Module):
    def __init__(self):
        super(CustomCNN, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1, groups=32)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1, groups=64)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1, groups=128)
        
        # Max pooling layer with 3x3 kernel
        self.pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        # Fully connected layers
        self.fc1 = nn.Linear(256 * 14 * 14, 512)  # Adjusted input size after pooling
        self.fc2 = nn.Linear(512, 256)        # Additional hidden layer
        self.fc3 = nn.Linear(256, 2)          # Binary classification layer

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.25)

        # Apply He Initialization to all layers
        self._initialize_weights()

    def _initialize_weights(self):
        # Loop through all layers in the module
        for m in self.modules():
            if isinstance(m, nn.Conv2d):  # For convolutional layers
                init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):  # For fully connected layers
                init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    init.zeros_(m.bias)

    def forward(self, x):
        # Apply convolutional layers with ReLU activation and max pooling
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = self.pool(self.relu(self.conv4(x)))
        
        # Flatten the tensor to feed into fully connected layers
        x = x.view(x.size(0), -1)
        
        # Apply fully connected layers with dropout and ReLU activation
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.dropout(self.relu(self.fc2(x)))  # Additional hidden layer
        x = self.fc3(x)  # Output layer
        return x

# Loss and Optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Train

#### Freezing This

In [20]:
# k = 5
# kfold = KFold(n_splits=k, shuffle=True)

# # Initialize results dictionary
# results = {}

# # Loop through k-folds
# for fold, (train_ids, val_ids) in enumerate(kfold.split(train_dataset)):
#     print(f'Fold {fold + 1}')
    
#     # Subset datasets for the current fold
#     train_subsampler = Subset(train_dataset, train_ids)
#     val_subsampler = Subset(train_dataset, val_ids)
    
#     # Create data loaders    
#     train_loader = DataLoader(train_subsampler, batch_size=32, shuffle=True)
#     val_loader = DataLoader(val_subsampler, batch_size=32, shuffle=False)
    
#     # Initialize the model
#     model = CustomCNN().to(device)
    
#     # Define loss function and optimizer
#     criterion = nn.CrossEntropyLoss()
#     optimizer = optim.Adam(model.parameters(), lr=0.001)
    
#     # Training loop
#     for epoch in range(10):  # Adjust number of epochs as needed
#         model.train()
#         running_loss = 0.0
#         for inputs, labels in train_loader:
#             optimizer.zero_grad()
#             outputs = model(inputs)
#             loss = criterion(outputs, labels)
#             loss.backward()
#             optimizer.step()
#             running_loss += loss.item()
        
#         print(f'Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}')
    
#     # Validation loop
#     model.eval()
#     correct = 0
#     total = 0
#     with torch.no_grad():
#         for inputs, labels in val_loader:
#             outputs = model(inputs)
#             _, predicted = torch.max(outputs, 1)
#             total += labels.size(0)
#             correct += (predicted == labels).sum().item()
    
#     # Save accuracy for the current fold
#     accuracy = 100 * correct / total
#     print(f'Fold {fold + 1}, Validation Accuracy: {accuracy:.2f}%')
#     results[fold] = accuracy

Fold 1
Epoch 1, Loss: 0.738676579102226
Epoch 2, Loss: 0.6100599804650182
Epoch 3, Loss: 0.4069567374561144
Epoch 4, Loss: 0.2181377453000649
Epoch 5, Loss: 0.14224674325922262
Epoch 6, Loss: 0.10155483003219833
Epoch 7, Loss: 0.07827273760314869
Epoch 8, Loss: 0.048444662121650967
Epoch 9, Loss: 0.061293622428227376
Epoch 10, Loss: 0.0553135316737968
Fold 1, Validation Accuracy: 95.11%
Fold 2
Epoch 1, Loss: 0.7375401154808376
Epoch 2, Loss: 0.6950695359188578
Epoch 3, Loss: 0.608172937579777
Epoch 4, Loss: 0.4746983051300049
Epoch 5, Loss: 0.3281523786161257
Epoch 6, Loss: 0.20465769003266873
Epoch 7, Loss: 0.18650074789057608
Epoch 8, Loss: 0.11212774414731108
Epoch 9, Loss: 0.099261822907821
Epoch 10, Loss: 0.07372033814697163
Fold 2, Validation Accuracy: 97.83%
Fold 3
Epoch 1, Loss: 0.7531616895095162
Epoch 2, Loss: 0.6818736921186033
Epoch 3, Loss: 0.5864553555198337
Epoch 4, Loss: 0.523631126984306
Epoch 5, Loss: 0.4691731592883234
Epoch 6, Loss: 0.42762855472772016
Epoch 7, Loss

### Model Evaluation 

#### Freezing This

In [22]:
# model.eval()
# with torch.no_grad():
#     correct = 0
#     total = 0

#     for images, labels in test_loader:
#         images, labels = images.to(device), labels.to(device)
#         outputs = model(images)
#         _, predicted = torch.max(outputs.data, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()

#     print(f"Test Accuracy: {100 * correct / total:.2f}%")

Test Accuracy: 75.00%


### Save and Load Model

In [30]:
# Save the Model
# torch.save(model.state_dict(), "covid_ct_model_ICP_K32_256_HL_1_2_512Neurons_3MxPl_Drpout_30_KFold.pth")

# Load the model
model = CustomCNN()
# model.load_state_dict(torch.load("covid_ct_model_ICP_K32_256_HL_1_2_512Neurons_3MxPl_Drpout_30_KFold.pth"))
model.eval()

# Predict on a single image
from PIL import Image

# Define the transform used during training
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the image
image_path = "/Users/rohanojha/Documents/01_Sem_1_DS 5220 Code/SML_Project/Code_Test/single_prediction/normal.png"  # Replace with the actual path
# image_path = "/Users/rohanojha/Documents/01_Sem_1_DS 5220 Code/SML_Project/Code_Test/single_prediction/covid.png"
# image_path = "/Users/rohanojha/Documents/01_Sem_1_DS 5220 Code/SML_Project/Code_Test/single_prediction/Non-Covid (10).png"
image = Image.open(image_path).convert('RGB')

# Apply transformations
image_tensor = transform(image).unsqueeze(0)  # Add batch dimension

# Move the tensor to the same device as the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
image_tensor = image_tensor.to(device)
model = model.to(device)

# Predict
with torch.no_grad():
    output = model(image_tensor)
    _, predicted_class = torch.max(output, 1)

# Map predicted class to label
classes = ["Normal", "Covid"]
prediction = classes[predicted_class.item()]
print(f"Prediction: {prediction}")

Prediction: Normal
